 # Question answering

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## Load the dataset

In [ ]:
from datasets import load_dataset
squad = load_dataset("squad", split="train[:5000]")

In [ ]:
squad = squad.train_test_split(test_size=0.2)

## Inspect data

In [ ]:
squad['train'][0]

In [ ]:
from transformers import AutoTokenizer
model_name = "distilbert-base-uncased"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

## Preprocess the data

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

## Data collator

In [ ]:
from transformers import DefaultDataCollator
data_collator = DefaultDataCollator()

## Model Training

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

## Configs

In [ ]:
output_model_name = 'questionanswering-distilbert-squad'
learning_rate = 2e-5
batch_size = 16
weight_decay = 0.01
epochs=3


In [ ]:
train_args = TrainingArguments(
    output_dir=output_model_name, 
    evaluation_strategy='epoch', 
    learning_rate=learning_rate, 
    per_device_train_batch_size=batch_size, 
    per_device_eval_batch_size=batch_size, 
    num_train_epochs=epochs, 
    weight_decay=weight_decay, 
    push_to_hub=True
)

In [ ]:
trainer = Trainer(
    model=model, 
    args=train_args, 
    train_dataset=tokenized_squad["train"], 
    eval_dataset=tokenized_squad["test"], 
    tokenizer=tokenizer, 
    data_collator=data_collator
)

In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub()

## Inference

In [ ]:
question = "How many programming languages does BLOOM support?"
context = "BLOOM has 176 billion parameters and can generate text in 46 natural languages and 13 programming languages"

In [ ]:
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained(f"StatsGary/{output_model_name}")
inputs = tokenizer(question, context, return_tensors='pt')

In [ ]:
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained(f"StatsGary/{output_model_name}")

In [ ]:
import torch
with torch.no_grad():
    outputs = model(**inputs)


In [ ]:
answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

In [ ]:
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)

## Inference function

In [ ]:
# def get_answer(model, context, question):
#     qa_pipeline = pipeline("question-answering", model=model)
#     print(qa_pipeline(question=question, context=context))

#     tokenizer

